# Exploring the Data


In [2]:
import pandas as pd

import os
for dirname, _, filenames in os.walk('../Data/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

../Data/test.csv
../Data/data.md
../Data/train.csv
../Data/transactions.csv
../Data/store-sales-time-series-forecasting.zip
../Data/oil.csv
../Data/holidays_events.csv
../Data/sample_submission.csv
../Data/stores.csv


In [3]:
df = pd.read_csv('../Data/train.csv')

# View the first and last few rows of the data
df.head(-5)


,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0
1,1,2013-01-01,1,BABY CARE,0.000,0
2,2,2013-01-01,1,BEAUTY,0.000,0
3,3,2013-01-01,1,BEVERAGES,0.000,0
4,4,2013-01-01,1,BOOKS,0.000,0
...,...,...,...,...,...,...
3000878,3000878,2017-08-15,9,MAGAZINES,11.000,0
3000879,3000879,2017-08-15,9,MEATS,449.228,0
3000880,3000880,2017-08-15,9,PERSONAL CARE,522.000,11
3000881,3000881,2017-08-15,9,PET SUPPLIES,6.000,0


In [6]:
# Get summary statistics for the data
df.describe()

,id,store_nbr,sales,onpromotion
count,3.000888e+06,3.000888e+06,3.000888e+06,3.000888e+06
mean,1.500444e+06,2.750000e+01,3.577757e+02,2.602770e+00
std,8.662819e+05,1.558579e+01,1.101998e+03,1.221888e+01
min,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,7.502218e+05,1.400000e+01,0.000000e+00,0.000000e+00
50%,1.500444e+06,2.750000e+01,1.100000e+01,0.000000e+00
75%,2.250665e+06,4.100000e+01,1.958473e+02,0.000000e+00
max,3.000887e+06,5.400000e+01,1.247170e+05,7.410000e+02


see how many nonzero entries there are

In [5]:
sum(df['sales']!=0)

2061758

## Try random forest

for this we have to masssage the data a bit

In [13]:
df2 = pd.read_csv("../Data/train.csv",
    index_col='id',
    parse_dates=['date'])
df2.head(-5)

,date,store_nbr,family,sales,onpromotion
id,,,,,
0,2013-01-01,1,AUTOMOTIVE,0.000,0
1,2013-01-01,1,BABY CARE,0.000,0
2,2013-01-01,1,BEAUTY,0.000,0
3,2013-01-01,1,BEVERAGES,0.000,0
4,2013-01-01,1,BOOKS,0.000,0
...,...,...,...,...,...
3000878,2017-08-15,9,MAGAZINES,11.000,0
3000879,2017-08-15,9,MEATS,449.228,0
3000880,2017-08-15,9,PERSONAL CARE,522.000,11


Define the target variable (the one we are going to predict) 

In [11]:
y = df2.sales

See which are the features we can use for the prediction (in principle we can use the variables that are numerical)

In [16]:
df2.columns

Index(['date', 'store_nbr', 'family', 'sales', 'onpromotion'], dtype='object')

In [24]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000888 entries, 0 to 3000887
Data columns (total 5 columns):
 #   Column       Dtype         
---  ------       -----         
 0   date         datetime64[ns]
 1   store_nbr    int64         
 2   family       object        
 3   sales        float64       
 4   onpromotion  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 137.4+ MB


first we need to make the date into an integer or float

In [35]:
# An example
from datetime import datetime
curr_dt = datetime.now()
print("Current datetime: ", curr_dt)
timestamp = int(round(curr_dt.timestamp()))
print("Integer timestamp of current datetime: ",
      timestamp)

Current datetime:  2023-05-08 16:45:23.936833
Integer timestamp of current datetime:  1683557124


In [ ]:
# apply to all
df2['date'] = df2['date'].map(lambda a : int(round(a.timestamp())) )

In [53]:
df2.head(-5)

,date,store_nbr,family,sales,onpromotion
id,,,,,
0,1356998400,1,AUTOMOTIVE,0.000,0
1,1356998400,1,BABY CARE,0.000,0
2,1356998400,1,BEAUTY,0.000,0
3,1356998400,1,BEVERAGES,0.000,0
4,1356998400,1,BOOKS,0.000,0
...,...,...,...,...,...
3000878,1502755200,9,MAGAZINES,11.000,0
3000879,1502755200,9,MEATS,449.228,0
3000880,1502755200,9,PERSONAL CARE,522.000,11


In [54]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000888 entries, 0 to 3000887
Data columns (total 5 columns):
 #   Column       Dtype  
---  ------       -----  
 0   date         int64  
 1   store_nbr    int64  
 2   family       object 
 3   sales        float64
 4   onpromotion  int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 201.9+ MB


lets see if we can make "family" into a number (todo)

In [65]:
features = ['date', 'store_nbr', 'onpromotion']
x = df2[features]
print(x)

               date  store_nbr  onpromotion
id                                         
0        1356998400          1            0
1        1356998400          1            0
2        1356998400          1            0
3        1356998400          1            0
4        1356998400          1            0
...             ...        ...          ...
3000883  1502755200          9            0
3000884  1502755200          9            1
3000885  1502755200          9          148
3000886  1502755200          9            8
3000887  1502755200          9            0

[3000888 rows x 3 columns]


In [66]:

from sklearn.tree import DecisionTreeRegressor

# Define model. Specify a number for random_state to ensure same results each run
tree_model = DecisionTreeRegressor(random_state=1)

# Fit model
tree_model.fit(x, y)

DecisionTreeRegressor(random_state=1)

### Test prediction

In [90]:
print("Making predictions for the following 15 items:")
print(df2.iloc[-215:-200,:])
print("The predictions are")
prediction = tree_model.predict(x.iloc[-215:-200,:])
print(prediction)

Making predictions for the following 15 items:
               date  store_nbr                   family     sales  onpromotion
id                                                                            
3000673  1502755200         52      HOME AND KITCHEN II    54.000            2
3000674  1502755200         52          HOME APPLIANCES     0.000            0
3000675  1502755200         52                HOME CARE   302.000            5
3000676  1502755200         52               LADIESWEAR     7.000            0
3000677  1502755200         52          LAWN AND GARDEN    54.000            0
3000678  1502755200         52                 LINGERIE     6.000            0
3000679  1502755200         52         LIQUOR,WINE,BEER   171.000            8
3000680  1502755200         52                MAGAZINES    19.000            0
3000681  1502755200         52                    MEATS   435.828            0
3000682  1502755200         52            PERSONAL CARE   278.000           12
30006

# Calculate Success rate

In [91]:
data = df2.iloc[-215:-200,3]
# data = data.reset_index(drop=True)
print(data)
print(prediction)

eff = (prediction - data)/data
print("Efficiency: (expressed as 100% distance from expected value) " , eff*100)

id
3000673      54.000
3000674       0.000
3000675     302.000
3000676       7.000
3000677      54.000
3000678       6.000
3000679     171.000
3000680      19.000
3000681     435.828
3000682     278.000
3000683       9.000
3000684      20.000
3000685     397.340
3000686     104.418
3000687    3803.368
Name: sales, dtype: float64
[  54.           68.50452632  302.           68.50452632   68.50452632
   68.50452632 1424.32633333   68.50452632   68.50452632  278.
   68.50452632   68.50452632   68.50452632   68.50452632 1424.32633333]
Efficiency: (expressed as 100% distance from expected value)  id
3000673       0.000000
3000674            inf
3000675       0.000000
3000676     878.636090
3000677      26.860234
3000678    1041.742105
3000679     732.939376
3000680     260.550139
3000681     -84.281752
3000682       0.000000
3000683     661.161404
3000684     242.522632
3000685     -82.759217
3000686     -34.393949
3000687     -62.550920
Name: sales, dtype: float64


out of 15 predictions: 3 predictions are spot-on, 2 are less than 50% off, the rest are off by up to 1000%